In [18]:
import os 
import json
import pandas as pd 
import numpy as np
import traceback
from dotenv import load_dotenv

In [19]:
load_dotenv()

True

In [2]:
from langchain.chat_models import ChatOpenAI

In [20]:
llm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], model_name="gpt-3.5-turbo",temperature=0.5)

In [23]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [26]:
RESPONSE_JSON = {
    "1": {
        "mcq" : "multiple choice question",
        "options" : {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq" : "multiple choice question",
        "options" : {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer",        
    },
    "3": {
        "mcq" : "multiple choice question",
        "options" : {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer", 
    },
}

In [24]:
TEMPLATE = """
Text:{text}
You are an expert MCQ Maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students\
    in {tone} tone.Make sure the questions are not repeated and check all the questions to be conforming as well.The questions should be well made\
    Make sure to format  your response like RESPONSE_JSON below and use it as a guide.\
    Ensure to make {number} MCQs
    ### Response_JSON
    {response_json}
"""

In [25]:
Quiz_prompt =  PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [30]:
quiz_chain = LLMChain(llm=llm,prompt=Quiz_prompt,output_key="quiz",verbose=True)

In [28]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Question for {subject} students.You need to evaluate the Complexity of the question \
    and give a complete analysis of the quiz. Only use at max 50 words for complexity. If the quiz is not at par with the cognitive and analytical abilities\
    of the students , update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.
    Quiz_MCQs : 
    {quiz}
        
    Check from an expert english writer of the above quiz.
"""

In [29]:
Quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [31]:
Quiz_evaluation_chain = LLMChain(llm=llm,prompt=Quiz_evaluation_prompt,output_key='review',verbose=True)

In [32]:
generation_chain = SequentialChain(chains=[quiz_chain,Quiz_evaluation_chain],input_variables=["text","number","subject","tone","response_json"],
                                   output_variables=["quiz","review"],verbose=True)